In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Dense,
    Dropout,
    LayerNormalization,
    MultiHeadAttention,
    Flatten,
    Add,
)
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

2023-10-12 14:47:04.756105: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.2 when it was built against 1.14.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
data = pd.read_csv("../data.csv")
data.head()

,Unnamed: 0.1,Unnamed: 0,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
0,0,2010-01-04,0.9133,1.4419,1.7200,1.0377,6.8273,5.1597,7.7555,46.27,92.55,3.396
1,1,2010-01-05,0.9143,1.4402,1.7296,1.0371,6.8258,5.1668,7.7564,46.13,91.48,3.385
2,2,2010-01-06,0.9189,1.4404,1.7292,1.0333,6.8272,5.1638,7.7546,45.72,92.53,3.379
3,3,2010-01-07,0.9168,1.4314,1.7409,1.0351,6.8280,5.1981,7.7539,45.67,93.31,3.368
4,4,2010-01-08,0.9218,1.4357,1.7342,1.0345,6.8274,5.1827,7.7553,45.50,92.70,3.375


In [3]:
data.drop("Unnamed: 0.1", axis=1, inplace=True)
data.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
data["Date"] = pd.to_datetime(data["Date"])
data.set_index("Date", inplace=True)
data.replace(0, np.nan, inplace=True)

In [4]:
data

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Date,,,,,,,,,,
2010-01-04,0.9133,1.4419,1.7200,1.0377,6.8273,5.1597,7.7555,46.27,92.55,3.3960
2010-01-05,0.9143,1.4402,1.7296,1.0371,6.8258,5.1668,7.7564,46.13,91.48,3.3850
2010-01-06,0.9189,1.4404,1.7292,1.0333,6.8272,5.1638,7.7546,45.72,92.53,3.3790
2010-01-07,0.9168,1.4314,1.7409,1.0351,6.8280,5.1981,7.7539,45.67,93.31,3.3680
2010-01-08,0.9218,1.4357,1.7342,1.0345,6.8274,5.1827,7.7553,45.50,92.70,3.3750
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.6978,1.1174,4.0507,1.3073,6.9954,6.6829,7.7874,71.45,109.47,4.1260
2019-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.interpolate(method="linear", limit_direction="forward")

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Date,,,,,,,,,,
2010-01-04,0.913300,1.441900,1.720000,1.0377,6.8273,5.1597,7.755500,46.27,92.550000,3.3960
2010-01-05,0.914300,1.440200,1.729600,1.0371,6.8258,5.1668,7.756400,46.13,91.480000,3.3850
2010-01-06,0.918900,1.440400,1.729200,1.0333,6.8272,5.1638,7.754600,45.72,92.530000,3.3790
2010-01-07,0.916800,1.431400,1.740900,1.0351,6.8280,5.1981,7.753900,45.67,93.310000,3.3680
2010-01-08,0.921800,1.435700,1.734200,1.0345,6.8274,5.1827,7.755300,45.50,92.700000,3.3750
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.697800,1.117400,4.050700,1.3073,6.9954,6.6829,7.787400,71.45,109.470000,4.1260
2019-12-28,0.698667,1.118833,4.038867,1.3068,6.9924,6.6749,7.786833,71.40,109.263333,4.1191
2019-12-29,0.699533,1.120267,4.027033,1.3063,6.9894,6.6669,7.786267,71.35,109.056667,4.1122


In [6]:
data.interpolate(method="linear", limit_direction="forward", inplace=True)
LOOK_BACK = 30
PREDICT_DAY = 1
SPLIT_RATIO = 0.8

In [7]:
def Create_Data(
    data, lookback=LOOK_BACK, pred_len=PREDICT_DAY, split_ratio=SPLIT_RATIO
):
    if lookback < 2:
        print("ERROR: Lookback too small")
        return -1

    # declarations

    x = {}
    y = {}
    xtr = {}
    xt = {}
    ytr = {}
    yt = {}
    scalers = {}

    # Creating stepped data

    for i in data.columns:
        xtemp = pd.DataFrame(data[i])
        for j in range(1, lookback + 1):
            xtemp[i + str(j)] = data[i].shift(-1 * j)
        x[i] = xtemp.dropna()

    # Splitting data into x and y

    for i in x.keys():
        y[i] = pd.DataFrame(x[i].iloc[:, -pred_len])
        x[i] = x[i].iloc[:, :-pred_len]

    # Normalizing x and y values

    for i in x.keys():
        scalers[i + "_x"] = MinMaxScaler(feature_range=(0, 1))
        x[i] = scalers[i + "_x"].fit_transform(x[i])
        scalers[i + "_y"] = MinMaxScaler(feature_range=(0, 1))
        y[i] = scalers[i + "_y"].fit_transform(y[i])

    # setting train and test sizes

    tr_len = int(split_ratio * y["India"].shape[0])
    t_len = y["India"].shape[0] - tr_len

    # creating training and testing data

    for i in x.keys():
        xtr[i] = x[i][:tr_len]
        ytr[i] = y[i][:tr_len]
        xt[i] = x[i][-t_len:]
        yt[i] = y[i][-t_len:]

    # returning pertinent data

    return x, y, xtr, xt, ytr, yt, scalers

In [8]:
x, y, xtr, xt, ytr, yt, scalers = Create_Data(data)

In [9]:
def transformer_encoder(sequence_length, feature_dim, num_transformer_blocks=3):
    inputs = Input(shape=(sequence_length, feature_dim))
    x = inputs

    for _ in range(num_transformer_blocks):
        # Transformer Block with multi-head attention
        query = LayerNormalization(epsilon=1e-6)(x)
        key = LayerNormalization(epsilon=1e-6)(x)
        value = LayerNormalization(epsilon=1e-6)(x)

        attn_output = MultiHeadAttention(num_heads=4, key_dim=feature_dim, dropout=0.3)(
            query, value, key=key
        )
        x = Add()([x, attn_output])  # Residual connection
        x = LayerNormalization(epsilon=1e-6)(x)

        # Feed Forward Network inside transformer
        ffn_output = Dense(128, activation="relu")(x)
        ffn_output = Dropout(0.3)(ffn_output)
        ffn_output = Dense(64, activation="relu")(ffn_output)
        ffn_output = Dense(feature_dim)(ffn_output)
        x = Add()([x, ffn_output])  # Residual connection
        x = LayerNormalization(epsilon=1e-6)(x)

    x = Flatten()(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation="relu")(x)
    outputs = Dense(PREDICT_DAY)(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss="mean_squared_error", optimizer="adam")
    return model


def Create_model(x, y, lookback=LOOK_BACK, Pred_size=PREDICT_DAY):
    models = {}
    for i in x.keys():
        reshaped_x = np.reshape(x[i], (x[i].shape[0], 1, x[i].shape[1]))
        models[i] = transformer_encoder(1, reshaped_x.shape[2])
        print(i)
    return models

In [10]:
m = Create_model(x, y)

2023-10-12 14:47:06.639552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-12 14:47:06.669064: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-12 14:47:06.669567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Australia
Europe
Brazil
Canada
China
Denmark
Hong Kong
India
Japan
Malaysia


In [11]:
def Execute_model(model, xtr, ytr, xt, yt, scaler):
    MAPE = {}
    MAE = {}
    MSE = {}
    for i in model.keys():
        print(i)
        reshaped_xtr = np.reshape(xtr[i], (xtr[i].shape[0], 1, xtr[i].shape[1]))
        reshaped_xt = np.reshape(xt[i], (xt[i].shape[0], 1, xt[i].shape[1]))

        es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10)
        reduce_lr = ReduceLROnPlateau(
            monitor="val_loss", factor=0.2, patience=5, min_lr=0.0001
        )

        model[i].fit(
            reshaped_xtr,
            ytr[i],
            epochs=100,
            batch_size=64,
            verbose=1,
            validation_split=0.2,
            callbacks=[es, reduce_lr],
        )

        temp = model[i].predict(reshaped_xt)
        pred = scaler[i + "_y"].inverse_transform(temp)
        act = scaler[i + "_y"].inverse_transform(yt[i])

        MSE[i] = mean_squared_error(act, pred)
        MAE[i] = mean_absolute_error(act, pred)
        MAPE[i] = mean_absolute_percentage_error(act, pred)

    results = pd.DataFrame([MSE, MAE, MAPE])
    results["Metric"] = ["MSE", "MAE", "MAPE"]
    results.set_index("Metric", inplace=True)
    return results

In [12]:
result = Execute_model(m, xtr, ytr, xt, yt, scalers)

Australia
Epoch 1/100


2023-10-12 14:47:16.882505: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f27e8aa6800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-12 14:47:16.882538: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2023-10-12 14:47:16.888224: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-12 14:47:16.901652: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-10-12 14:47:17.035625: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


37/37 [==============================] - 14s 45ms/step - loss: 0.2110 - val_loss: 0.0193 - lr: 0.0010
Epoch 2/100
37/37 [==============================] - 1s 28ms/step - loss: 0.0985 - val_loss: 0.0143 - lr: 0.0010
Epoch 3/100
37/37 [==============================] - 1s 27ms/step - loss: 0.0935 - val_loss: 0.0175 - lr: 0.0010
Epoch 4/100
37/37 [==============================] - 1s 23ms/step - loss: 0.0889 - val_loss: 0.0247 - lr: 0.0010
Epoch 5/100
37/37 [==============================] - 1s 25ms/step - loss: 0.0813 - val_loss: 0.0279 - lr: 0.0010
Epoch 6/100
37/37 [==============================] - 1s 25ms/step - loss: 0.0804 - val_loss: 0.0181 - lr: 0.0010
Epoch 7/100
37/37 [==============================] - 1s 29ms/step - loss: 0.0801 - val_loss: 0.0190 - lr: 0.0010
Epoch 8/100
37/37 [==============================] - 1s 29ms/step - loss: 0.0774 - val_loss: 0.0275 - lr: 2.0000e-04
Epoch 9/100
37/37 [==============================] - 1s 29ms/step - loss: 0.0788 - val_loss: 0.0276 - l

In [13]:
result

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Metric,,,,,,,,,,
MSE,0.013630,0.004645,2.383435,0.057610,0.229974,0.606214,0.005757,208.875711,250.894018,0.676819
MAE,0.110948,0.062224,1.517548,0.237970,0.417746,0.744211,0.074639,14.205794,15.710139,0.814691
MAPE,0.156324,0.054959,0.395891,0.181148,0.060507,0.113424,0.009522,0.203452,0.142949,0.198886


In [14]:
import json
import os

# Define the filename
filename = "../results.json"

# Check if the file exists
if os.path.isfile(filename):
    # If the file exists, load the existing data
    with open(filename, "r") as f:
        data = json.load(f)
else:
    # If the file doesn't exist, create an empty dictionary
    data = {}

# Add the result to the dictionary
data["Transformer(Soft Attention)"] = result.to_dict()

# Write the dictionary to the file
with open(filename, "w") as f:
    json.dump(data, f)